In [1]:
# Install unrar
!apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [2]:

# Unzip the rar file
!unrar x /content/mimic-iv-ext-direct-1.0.0.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/mimic-iv-ext-direct-1.0.0.rar

Creating    mimic-iv-ext-direct-1.0.0                                 OK
Creating    mimic-iv-ext-direct-1.0.0/mimic-iv-ext-direct-1.0.0       OK
Extracting  mimic-iv-ext-direct-1.0.0/mimic-iv-ext-direct-1.0.0/.DS_Store       0%  OK 
Creating    mimic-iv-ext-direct-1.0.0/mimic-iv-ext-direct-1.0.0/diagnostic_kg  OK
Creating    mimic-iv-ext-direct-1.0.0/mimic-iv-ext-direct-1.0.0/diagnostic_kg/Diagnosis_flowchart  OK
Extracting  mimic-iv-ext-direct-1.0.0/mimic-iv-ext-direct-1.0.0/diagnostic_kg/Diagnosis_flowchart/Acute Coronary Syndrome.json       0%  OK 
Extracting  mimic-iv-ext-direct-1.0.0/mimic-iv-ext-direct-1.0.0/diagnostic_kg/Diagnosis_flowchart/Adrenal Insufficiency.json       0%  OK 
Extracting  mimic-iv-ext-direct-1.0.0/mimic-iv-ext-direct-1.0.0/diagnostic_kg/Diagnosis_flowchart/Alzheimer.json       0%  OK 
Extract

In [3]:
import os
import json
import pandas as pd

class MIMICDataLoader:
    def __init__(self):
        pass

    def load_and_preprocess(self, data_path):
        all_notes = []
        for root, dirs, files in os.walk(data_path):
            for file in files:
                if file.endswith('.json'):
                    file_path = os.path.join(root, file)
                    try:
                        with open(file_path, 'r') as f:
                            data = json.load(f)
                            # Inspect a sample JSON file's keys
                            if len(all_notes) < 5: # Only print for first 5 files
                                print(f"\n--- Inspecting {file} ---")
                                print(f"Keys in JSON: {list(data.keys())}")
                                print(f"Diagnosis value: {data.get('Diagnosis')}")
                                print(f"Treatment value: {data.get('Treatment')}")
                                print("-----------------------")

                            # --- Modified data extraction logic ---
                            keys = list(data.keys())
                            if not keys:
                                continue # Skip empty JSONs

                            # Assuming the first key contains diagnosis information (e.g., 'Acute Coronary Syndrome$NSTEMI')
                            main_key = keys[0]
                            diagnosis_raw = main_key.split('$')[0].replace('_', ' ')
                            diagnosis = diagnosis_raw if diagnosis_raw else ''

                            # Concatenate input fields for document text, chief complaint, history, treatment
                            chief_complaint = data.get('input1', '')
                            history = data.get('input2', '')
                            treatment = data.get('input3', '') # Arbitrary mapping for demonstration

                            document_text_parts = [data.get(f'input{i}', '') for i in range(1, 7)]
                            document_text = f"Diagnosis: {diagnosis} | Chief Complaint: {chief_complaint} | History: {history} | Treatment: {treatment} | Notes: {' '.join(document_text_parts)}"

                            all_notes.append({
                                'document_id': file,
                                'document_text': document_text,
                                'chief_complaint': chief_complaint,
                                'history': history,
                                'diagnosis': diagnosis,
                                'treatment': treatment
                            })
                    except json.JSONDecodeError:
                        print(f"Error decoding JSON from {file_path}")
                    except Exception as e:
                        print(f"An error occurred while processing {file_path}: {e}")
        return pd.DataFrame(all_notes)

# Instantiate the data loader
data_loader = MIMICDataLoader()

# Set the data path to the unzipped directory
# Assuming the unrar command extracted the contents to /content/mimic-iv-ext-direct-1.0.0
data_path = '/content/mimic-iv-ext-direct-1.0.0/mimic-iv-ext-direct-1.0.0'

# Load and preprocess the data
original_notes_df = data_loader.load_and_preprocess(data_path)

# Filter out documents with empty 'diagnosis' and 'treatment' fields
# Now this filter should retain documents if the new parsing logic works
notes_df = original_notes_df[(original_notes_df['diagnosis'].str.strip() != '') | (original_notes_df['treatment'].str.strip() != '')]

print(f"Original documents: {len(original_notes_df)}")
print(f"Filtered documents: {len(notes_df)}")
print("Data loading and filtering complete. First 5 rows of filtered data:")
print(notes_df.head())


--- Inspecting 18427803-DS-5.json ---
Keys in JSON: ['Migraine With Aura$Intermedia_3', 'input1', 'input2', 'input3', 'input4', 'input5', 'input6']
Diagnosis value: None
Treatment value: None
-----------------------

--- Inspecting 18805216-DS-21.json ---
Keys in JSON: ['Migraine With Aura$Intermedia_3', 'input1', 'input2', 'input3', 'input4', 'input5', 'input6']
Diagnosis value: None
Treatment value: None
-----------------------

--- Inspecting 17676552-DS-10.json ---
Keys in JSON: ['Migraine With Aura$Intermedia_3', 'input1', 'input2', 'input3', 'input4', 'input5', 'input6']
Diagnosis value: None
Treatment value: None
-----------------------

--- Inspecting 17185323-DS-14.json ---
Keys in JSON: ['Migraine Without Aura$Intermedia_3', 'input1', 'input2', 'input3', 'input4', 'input5', 'input6']
Diagnosis value: None
Treatment value: None
-----------------------

--- Inspecting 14946203-DS-21.json ---
Keys in JSON: ['UA$Intermedia_5', 'input1', 'input2', 'input3', 'input4', 'input5', 'i

In [4]:
!pip install faiss-cpu
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

class ClinicalRetriever:
    def __init__(self):
        self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        self.index = None
        self.documents = None
        self.metadata = None

    def build_index(self, documents, metadata):
        self.documents = documents
        self.metadata = metadata
        embeddings = self.model.encode(documents, show_progress_bar=True, batch_size=32)
        embeddings = np.array(embeddings).astype('float32')
        self.index = faiss.IndexFlatIP(embeddings.shape[1])
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings)

    def retrieve(self, query, top_k=5):
        query_emb = self.model.encode([query])
        query_emb = np.array(query_emb).astype('float32')
        faiss.normalize_L2(query_emb)
        scores, indices = self.index.search(query_emb, top_k)
        return [(self.documents[idx], self.metadata[idx], float(scores[0][i]))
                for i, idx in enumerate(indices[0]) if idx < len(self.documents)]

retriever = ClinicalRetriever()
retriever.build_index(notes_df['document_text'].tolist(), notes_df.to_dict('records'))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 111.8 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

In [5]:
results = retriever.retrieve("chest pain treatment", top_k=3)
for i, (doc, meta, score) in enumerate(results, 1):
    print(f"{i}. Score: {score:.3f}\n{doc[:150]}...\n")

1. Score: 0.611
Diagnosis: UA | Chief Complaint: Chest Pain
 | History: He is a gentleman with history of HTN, HLD, PVD, tobacco use presenting with two weeks of inte...

2. Score: 0.588
Diagnosis: UA | Chief Complaint: Chest pain
 | History: He is a male with a history of Aortic stenosis (s/p bioprosthetic AVR and CAD (s/p CABG nd DES...

3. Score: 0.560
Diagnosis: Type A Aortic Dissection | Chief Complaint: chest pain
 | History: He is a 67 year old man with no significant past medical history. He beg...



In [6]:
class ClinicalGenerator:
    def generate_response(self, query, retrieved_docs):
        response = f"Query: '{query}'\n\n"
        for i, (doc, meta, score) in enumerate(retrieved_docs, 1):
            response += f"{i}. Relevance: {score:.2f}\n"
            if 'diagnosis' in meta:
                response += f"   Diagnosis: {meta['diagnosis']}\n"
            if 'treatment' in meta:
                response += f"   Treatment: {meta['treatment'][:100]}...\n\n"
        return response

generator = ClinicalGenerator()

In [7]:
from datetime import datetime

class ClinicalRAGSystem:
    def __init__(self, retriever, generator, df):
        self.retriever = retriever
        self.generator = generator
        self.df = df

    def query(self, query, top_k=3):
        docs = self.retriever.retrieve(query, top_k)
        response = self.generator.generate_response(query, docs)
        return {
            'query': query,
            'response': response,
            'retrieved_documents': [{'text': d, 'metadata': m, 'score': s} for d, m, s in docs],
            'timestamp': datetime.now().isoformat()
        }

rag_system = ClinicalRAGSystem(retriever, generator, notes_df)

In [8]:
queries = [
    "diabetes treatment",
    "hypertension diagnosis",
    "stroke symptoms",
    "asthma management",
    "sepsis treatment",
    "heart attack signs",
    "kidney stone symptoms",
    "COPD exacerbation",
    "liver failure treatment",
    "seizure management"
]

for q in queries:
    result = rag_system.query(q, 3)
    print("="*80)
    print(result['response'])
    print("="*80 + "\n")

Query: 'diabetes treatment'

1. Relevance: 0.47
   Diagnosis: Type II diabetes
   Treatment: 1. CARDIAC RISK FACTORS  
+ Hypertension  
+ Dyslipidemia  
+ Coronary artery disease
2. CARDIAC HIS...

2. Relevance: 0.45
   Diagnosis: Type II diabetes
   Treatment: rheumatoid arthritis   
hypertension  
s/p abdominal surgery for GSW
...

3. Relevance: 0.45
   Diagnosis: Type I Diabetes
   Treatment: +Previous admissions for nausea/vomiting with h/o esophagitis  and with concern for diabetic gastrop...



Query: 'hypertension diagnosis'

1. Relevance: 0.58
   Diagnosis: Hypertension
   Treatment: +Paroxysmal atrial fibrillation  
+Dyslipidemia
+Tachy/brady s/p PPM ___ (syncopal episodes)
+Osteop...

2. Relevance: 0.57
   Diagnosis: Hypertension
   Treatment: None
...

3. Relevance: 0.55
   Diagnosis: Hypertension
   Treatment: +HCV
+Hypothyroidism
+h/o pneumonia
+MR/CHF - in the setting of pneumonia  
+Preeclampsia  
+Anxiety...



Query: 'stroke symptoms'

1. Relevance: 0.57
   Diagnosis: 

In [9]:
import time

# Speed test
times = []
for _ in range(10):
    start_time = time.time()
    # Call retriever.retrieve and discard its return value for timing purposes
    _ = retriever.retrieve("test", 5)
    end_time = time.time()
    times.append(end_time - start_time)

print(f"Avg retrieval time: {np.mean(times):.4f}s")

# Relevance scores
results = [rag_system.query(q, 3) for q in queries]
scores = [d['score'] for r in results for d in r['retrieved_documents']]
print(f"Mean score: {np.mean(scores):.3f}")

Avg retrieval time: 0.0095s
Mean score: 0.488


In [10]:
import json
notes_df.to_csv('processed_notes.csv', index=False)
with open('results.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)

**Integrating Generative model**

In [11]:
# Install
!pip install -q transformers torch

# Generator (FREE - Hugging Face)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

class HuggingFaceGenerator:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
        self.model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)

    def generate_response(self, query, retrieved_docs):
        context = "\n".join([f"Doc{i}: {doc[:200]}" for i, (doc,_,_) in enumerate(retrieved_docs,1)])
        prompt = f"Answer medical question.\n\nRecords: {context}\n\nQuestion: {query}\n\nAnswer:"
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(self.device)
        outputs = self.model.generate(**inputs, max_length=300, num_beams=4)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    # New method for direct text generation without RAG context
    def generate_pure_text(self, prompt_text, max_length=150, num_beams=4):
        inputs = self.tokenizer(prompt_text, return_tensors="pt", max_length=512, truncation=True).to(self.device)
        outputs = self.model.generate(**inputs, max_length=max_length, num_beams=num_beams)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

generator = HuggingFaceGenerator()


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
from datetime import datetime

class EnhancedRAGSystem:
    def __init__(self, retriever, generator, df):
        self.retriever = retriever
        self.generator = generator
        self.df = df

    def query(self, query, top_k=3):
        docs = self.retriever.retrieve(query, top_k)
        response = self.generator.generate_response(query, docs)
        return {
            'query': query,
            'response': response,
            'retrieved_documents': [{'text': d, 'metadata': m, 'score': s} for d,m,s in docs],
            'timestamp': datetime.now().isoformat()
        }

enhanced_rag = EnhancedRAGSystem(retriever, generator, notes_df)

In [13]:
queries = ["diabetes treatment", "stroke symptoms", "asthma management", "sepsis treatment"]

for q in queries:
    result = enhanced_rag.query(q, 3)
    print("="*80)
    print(f"Query: {q}\n\n{result['response']}")
    print("="*80 + "\n")

Query: diabetes treatment

metformin, rheumatoid arthritis, and HTN, who p/w two days of episodic weakness, dizzyness, and dia

Query: stroke symptoms

Nausea and Weakness | History: HPI: ___ RH F who was in her usual, independent state of health until this morning. Around noon, she suddenly felt sick to Doc2: Diagnosis: Ischemic Stroke | Chief Complaint: None | History: Mr. ___ is a ___ white male with multiple medical problems including CVA ___, L sided weakness, uses walker, antigravity in arm at baseli Doc3: Diagnosis: Ischemic Stroke | Chief Complaint: None | History: Mr. ___ is a ___ white male with multiple medical problems including CVA ___, L sided weakness, uses walker, antigravity in arm at baseli

Query: asthma management

Doc1: Female with history of asthma on 3L O2 for ambulation, OSA on CPAP, allergic rhinitis, atopic dermatits, HTN, diabetes, anxiety and depre Doc2: Diagnosis: Severe Asthma | Chief Complaint: asthma | History: Female with history of asthma on 3L O2 for 

In [14]:
# -----------------------------------------
# IMPORTS
# -----------------------------------------
import gradio as gr
from datetime import datetime

# -----------------------------------------
# YOUR EXISTING RAG SYSTEM
# -----------------------------------------
# Re-define or ensure this is accessible from previous cells if it's not global
class EnhancedRAGSystem:
    def __init__(self, retriever, generator, df):
        self.retriever = retriever
        self.generator = generator
        self.df = df

    def query(self, query, top_k=3):
        docs = self.retriever.retrieve(query, top_k)
        response = self.generator.generate_response(query, docs)
        return {
            'query': query,
            'response': response,
            'retrieved_documents': [
                {'text': d, 'metadata': m, 'score': s} for d, m, s in docs
            ],
            'timestamp': datetime.now().isoformat()
        }

# Ensure 'retriever', 'generator', 'notes_df' are defined in prior cells
enhanced_rag = EnhancedRAGSystem(retriever, generator, notes_df)


# ---------------------------------------------------
# 1) NORMALIZER: Convert messy user input → clean query
# ---------------------------------------------------
def normalize_query(user_input):
    user_input = user_input.lower()

    if "diabetes" in user_input:
        return "diabetes treatment"
    if "stroke" in user_input:
        return "stroke symptoms"
    if "asthma" in user_input:
    	return "asthma management"
    if "sepsis" in user_input:
        return "sepsis treatment"

    # fallback → use raw query as-is
    return user_input


# ---------------------------------------------------
# 2) FUNCTION CONNECTED TO GRADIO
# ---------------------------------------------------
def rag_pipeline(user_input):
    # Convert user free-style question to your standard format
    normalized = normalize_query(user_input)

    # Run the RAG system, retrieving the top 3 documents
    result = enhanced_rag.query(normalized, top_k=3)

    # Extract diagnosis if available from the most relevant document
    disease_name = "N/A"
    if result["retrieved_documents"] and 'diagnosis' in result["retrieved_documents"][0]['metadata']:
        disease_name = result["retrieved_documents"][0]['metadata']['diagnosis']

    # Prepare pretty output
    final_answer = f"""
### 🔍 Normalized Query:
**{normalized}**

### 💡 Answer:
**Disease:** {disease_name}
{result['response']}

---

### 📄 Most Relevant Documents:
"""
    # Display all retrieved documents
    if result["retrieved_documents"]:
        for i, doc in enumerate(result["retrieved_documents"]):
            final_answer += f"""
**{i+1}. Score:** {doc['score']:.4f}
**Text:** {doc['text'][:300]}...

"""

    return final_answer


# ---------------------------------------------------
# 3) GRADIO UI
# ---------------------------------------------------
ui = gr.Interface(
    fn=rag_pipeline,
    inputs=gr.Textbox(lines=3, label="Ask a medical question"),
    outputs=gr.Markdown(label="RAG Answer"),
    title="Enhanced Medical RAG System",
    description="Type any medical question. The system will normalize your query, retrieve relevant notes, and generate an answer."
)

# RUN
ui.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://85961e19926b965c3d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Evaluation and Analysis**

In [15]:
def build_ground_truth(dataset):
    keywords = {}

    keywords["diabetes treatment"] = dataset[dataset["document_text"].str.contains("diab|glucose|insulin", regex=True)]["document_text"].tolist()
    keywords["stroke symptoms"]    = dataset[dataset["document_text"].str.contains("stroke|cva|weak|numb|aphasia", regex=True)]["document_text"].tolist()
    keywords["asthma management"]  = dataset[dataset["document_text"].str.contains("asthma|broncho|inhaler|airway", regex=True)]["document_text"].tolist()
    keywords["sepsis treatment"]   = dataset[dataset["document_text"].str.contains("sepsis|lactate|infection|shock", regex=True)]["document_text"].tolist()

    return keywords

ground_truth_docs = build_ground_truth(notes_df)


In [16]:
from sentence_transformers import util

def evaluate_retrieval_semantic(query, retrieved_docs, full_df, embed_model, threshold=0.40):

    # embed the query
    q_emb = embed_model.encode(query, convert_to_tensor=True)

    # compute similarity with ALL documents
    all_embs = embed_model.encode(full_df["clean_text"].tolist(), convert_to_tensor=True)
    sims = util.cos_sim(q_emb, all_embs)[0]

    # relevant docs = similarity above threshold
    relevant_indices = (sims > threshold).nonzero().flatten().tolist()
    total_relevant = len(relevant_indices)

    # compute relevance of retrieved docs
    retrieved_indices = [d[1]["row_id"] for d in retrieved_docs]
    relevant_retrieved = len(set(retrieved_indices) & set(relevant_indices))

    precision = relevant_retrieved / len(retrieved_docs)
    recall = relevant_retrieved / total_relevant if total_relevant > 0 else 0

    return precision, recall


In [17]:
def grade_answer(query, answer, generator):
    # The evaluation prompt for the generative model
    eval_prompt_template = f"""
Evaluate the clinical answer on a scale of 1—5.

Question: {query}
Answer: {answer}

Rate:
Relevance (1-5)
Coherence (1-5)
Accuracy (1-5)

Return EXACTLY like this:
Relevance: X
Coherence: Y
Accuracy: Z
"""

    # Use the new generate_pure_text method for direct evaluation
    # Increase max_length to allow the model to generate the full evaluation.
    output = generator.generate_pure_text(eval_prompt_template, max_length=100)

    print(f"\n--- Generator Output for Query '{query[:50]}...' ---")
    print(output)
    print(f"--- End Generator Output ---\n")

    import re
    # Using re.IGNORECASE to be more flexible, just in case.
    rel = re.search(r"Relevance:\s*(\d)", output, re.IGNORECASE)
    coh = re.search(r"Coherence:\s*(\d)", output, re.IGNORECASE)
    acc = re.search(r"Accuracy:\s*(\d)", output, re.IGNORECASE)

    if rel and coh and acc:
        return int(rel.group(1)), int(coh.group(1)), int(acc.group(1))

    # Fallback if regex fails to find matches
    return (3,3,3)  # fallback


In [18]:
import pandas as pd

eval_rows = []

# Add 'row_id' to notes_df for retrieval evaluation
# Also create a 'clean_text' column for semantic evaluation if it doesn't exist
if 'row_id' not in notes_df.columns:
    notes_df['row_id'] = notes_df.index
if 'clean_text' not in notes_df.columns:
    notes_df['clean_text'] = notes_df['document_text']

# Rebuild the retriever's index with the updated notes_df
# This ensures the metadata includes 'row_id' and 'clean_text' for evaluation
retriever.build_index(notes_df['document_text'].tolist(), notes_df.to_dict('records'))

for query_text, gt_docs in ground_truth_docs.items():

    # Retrieve top-k docs
    retrieved_docs = retriever.retrieve(query_text, top_k=4)

    # Evaluate retrieval (semantic)
    precision, recall = evaluate_retrieval_semantic(
        query_text,
        retrieved_docs,
        notes_df, # Use notes_df as the full_df for evaluation
        retriever.model
    )

    # Generate answer
    rag_output = enhanced_rag.query(query_text, top_k=4)
    answer = rag_output["response"]

    # Evaluate generation
    rel, coh, acc = grade_answer(query_text, answer, generator) # Use grade_answer and pass generator

    # Append row properly
    eval_rows.append({
        "Query": query_text,
        "Precision": precision,
        "Recall": recall,
        "Relevance Score": rel,
        "Coherence Score": coh,
        "Accuracy Score": acc,
        "Final Gen Score": (rel + coh + acc) / 3
    })

# Convert to DataFrame
df_eval = pd.DataFrame(eval_rows)
display(df_eval)

Batches:   0%|          | 0/17 [00:00<?, ?it/s]


--- Generator Output for Query 'diabetes treatment...' ---
1
--- End Generator Output ---


--- Generator Output for Query 'stroke symptoms...' ---
1
--- End Generator Output ---


--- Generator Output for Query 'asthma management...' ---
1
--- End Generator Output ---


--- Generator Output for Query 'sepsis treatment...' ---
1
--- End Generator Output ---



,Query,Precision,Recall,Relevance Score,Coherence Score,Accuracy Score,Final Gen Score
0,diabetes treatment,1.00,0.500000,3,3,3,3.0
1,stroke symptoms,1.00,0.142857,3,3,3,3.0
2,asthma management,1.00,0.400000,3,3,3,3.0
3,sepsis treatment,0.25,1.000000,3,3,3,3.0
